In [1]:
import numpy as np
import tensorflow as tf
import input_data
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [2]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_steps = 1

#Network Parameters
n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

In [3]:
# tf graph input
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder("float",[None,n_input])
y = tf.compat.v1.placeholder("float",[None,n_classes])


In [4]:
weights = {
    "h1":tf.Variable(tf.compat.v1.random_normal([n_input,n_hidden_1])),
    "h2":tf.Variable(tf.compat.v1.random_normal([n_hidden_1,n_hidden_2])),
    "out":tf.Variable(tf.compat.v1.random_normal([n_hidden_2,n_classes]))
          
          }

biases = {
    "b1":tf.Variable(tf.compat.v1.random_normal([n_hidden_1])),
    "b2":tf.Variable(tf.compat.v1.random_normal([n_hidden_2])),
    "out":tf.Variable(tf.compat.v1.random_normal([n_classes]))
}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
# Multiplayer Perceptron

def multilayer_perceptron(x):
    # hidden fully connected layer with 256 neurons
    layer1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer2 = tf.add(tf.matmul(layer1,weights['h2']),biases['b2'])
    out_layer = tf.add(tf.matmul(layer2,weights['out']),biases['out'])
    
    return out_layer

In [8]:
logit = multilayer_perceptron(X)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
logits=logit,labels=y))

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)
init = tf.compat.v1.global_variables_initializer()

In [9]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        #loop over all batch
        for i in range(total_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            _,c = sess.run([train_op,loss],feed_dict = {X:batch_x,y:batch_y})
            
            #compute avg loss
            avg_cost += c/total_batch 
        if epoch % display_steps ==0:
            print("Epoch : {}, Cost {}".format(epoch+1,avg_cost))
    print("Done With Optimization")

Epoch : 1, Cost 318.0158358348497
Epoch : 2, Cost 102.1064302513816
Epoch : 3, Cost 74.31456960157915
Epoch : 4, Cost 60.141673303083884
Epoch : 5, Cost 50.67453061970802
Epoch : 6, Cost 44.862774529457084
Epoch : 7, Cost 40.80014093399053
Epoch : 8, Cost 36.33177082278513
Epoch : 9, Cost 33.57272273237055
Epoch : 10, Cost 30.7395123837211
Epoch : 11, Cost 28.868015602285215
Epoch : 12, Cost 26.857013228156347
Epoch : 13, Cost 25.768157681551873
Epoch : 14, Cost 24.970954780578616
Epoch : 15, Cost 23.649149678620432
Done With Optimization


# Multilayer Perceptron With Keras

In [16]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

In [17]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [18]:
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0],num_pixels).astype("float32")
X_test = X_test.reshape(X_test.shape[0],num_pixels).astype("float32")
print(num_pixels)
print(X_train.shape)
print(X_test.shape)

784
(60000, 784)
(10000, 784)


In [19]:
# normalize input from 0-255 to 0-1

X_train = X_train / 255
X_test = X_test / 255

#oe hot encode output

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
print(num_classes)

10


In [14]:
# Hidden Layer with 784 neurons

In [20]:
def baseline_model():
    model = Sequential()
    model.add(Dense(num_pixels,input_dim=num_pixels,kernel_initializer="normal",activation="relu"))
    model.add(Dense(num_classes,kernel_initializer="normal",activation="softmax"))
    
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])
    return model

In [21]:
model = baseline_model()
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10, batch_size=200,verbose=2)
# Evaluation of model

scores = model.evaluate(X_test,y_test,verbose=0)
print("Baseline Error : {}".format(100-scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 4s - loss: 0.2771 - accuracy: 0.9201 - val_loss: 0.1347 - val_accuracy: 0.9585
Epoch 2/10
 - 3s - loss: 0.1094 - accuracy: 0.9681 - val_loss: 0.0986 - val_accuracy: 0.9710
Epoch 3/10
 - 3s - loss: 0.0700 - accuracy: 0.9794 - val_loss: 0.0754 - val_accuracy: 0.9758
Epoch 4/10
 - 4s - loss: 0.0490 - accuracy: 0.9860 - val_loss: 0.0764 - val_accuracy: 0.9756
Epoch 5/10
 - 3s - loss: 0.0355 - accuracy: 0.9899 - val_loss: 0.0644 - val_accuracy: 0.9796
Epoch 6/10
 - 3s - loss: 0.0262 - accuracy: 0.9933 - val_loss: 0.0570 - val_accuracy: 0.9821
Epoch 7/10
 - 3s - loss: 0.0187 - accuracy: 0.9955 - val_loss: 0.0645 - val_accuracy: 0.9794
Epoch 8/10
 - 3s - loss: 0.0150 - accuracy: 0.9964 - val_loss: 0.0624 - val_accuracy: 0.9800
Epoch 9/10
 - 3s - loss: 0.0108 - accuracy: 0.9977 - val_loss: 0.0632 - val_accuracy: 0.9810
Epoch 10/10
 - 3s - loss: 0.0079 - accuracy: 0.9987 - val_loss: 0.0560 - val_accuracy: 0.9823
Baseline Error : 1.

In [30]:
print(X_test[0].shape)
prediction = model.predict(X_test)

(784,)


In [46]:
print(np.round(prediction[5],1))
print(y_test[5])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
